In [ ]:
!pip install kaggle

In [ ]:
# This authenticates with Kaggle using the api key you put in your google drive
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [ ]:
!kaggle competitions download --force -c titanic
!kaggle competitions download -c titanic

403 - Forbidden
403 - Forbidden


In [ ]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 23.5MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.2MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.82MB/s]


In [ ]:
import pandas as pd
train_df_raw = pd.read_csv('/content/train.csv')
test_df_raw = pd.read_csv('/content/test.csv')

In [ ]:
train_df_raw


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
def do_transformations(t_df, df):
  df = df.drop("Ticket", axis=1)
  df = df.drop("Cabin", axis=1)
  df = df.drop("Name", axis=1)
  df = pd.get_dummies(df, ['Sex','Embarked'])
  df.Age = df.Age.fillna(t_df.Age.value_counts().index[0])
  df.Fare = df.Fare.fillna(t_df.Fare.value_counts().index[0])
  
  return df

In [ ]:
train_df = do_transformations(train_df_raw, train_df_raw)
test_df = do_transformations(train_df_raw, test_df_raw)

train_df = train_df.drop("PassengerId", axis=1)

In [ ]:
train_df

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,0,1,0,0,1
887,1,1,19.0,0,0,30.0000,1,0,0,0,1
888,0,3,24.0,1,2,23.4500,1,0,0,0,1
889,1,1,26.0,0,0,30.0000,0,1,1,0,0


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score

skf = KFold(n_splits=5, shuffle=True)

features = train_df[train_df.columns[1:]]
labels = train_df["Survived"]

total_preds = []
total_labels = []

for train_indices, test_indices in skf.split(features, labels):
    train_features = features.iloc[train_indices, :]
    train_labels = labels.iloc[train_indices]

    test_features = features.iloc[test_indices, :]
    test_labels = labels.iloc[test_indices]

    tree = DecisionTreeClassifier(max_depth=4)
    tree = tree.fit(train_features, train_labels)
    preds = tree.predict(test_features)
    
    total_preds.extend(preds)
    total_labels.extend(test_labels)

print(accuracy_score(total_labels, total_preds))
print(precision_score(total_labels, total_preds))
print(recall_score(total_labels, total_preds))

best_model = tree

0.8159371492704826
0.8272058823529411
0.6578947368421053


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

total_preds = []
total_labels = []

for train_indices, test_indices in skf.split(features, labels):
    train_features = features.iloc[train_indices, :]
    train_labels = labels.iloc[train_indices]

    test_features = features.iloc[test_indices, :]
    test_labels = labels.iloc[test_indices]

    tree = RandomForestClassifier(max_depth=4)

    tree = tree.fit(train_features, train_labels)
    preds = tree.predict(test_features)
    
    total_preds.extend(preds)
    total_labels.extend(test_labels)
    

print(accuracy_score(total_labels, total_preds))
print(precision_score(total_labels, total_preds))
print(recall_score(total_labels, total_preds))

0.8170594837261503
0.8118466898954704
0.6812865497076024


In [ ]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import r2_score
total_preds = []
total_labels = []

for train_indices, test_indices in skf.split(features, labels):
    train_features = features.iloc[train_indices, :]
    train_labels = labels.iloc[train_indices]

    test_features = features.iloc[test_indices, :]
    test_labels = labels.iloc[test_indices]

    clf = Ridge(alpha=1)

    clf = clf.fit(train_features, train_labels)
    preds = tree.predict(test_features)
    
    total_preds.extend(preds)
    total_labels.extend(test_labels)
    

print(r2_score(total_labels, total_preds))


0.2881794650560827


In [ ]:
def generate_submission_file(best_model, test_df):
  test_df2 = test_df.drop('PassengerId', axis=1)
  Y = best_model.predict(test_df2)
  
  
  submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": Y
  })
  submission.to_csv('/content/submission.csv', index=False)

In [ ]:
generate_submission_file(best_model, test_df)

In [ ]:
!kaggle competitions submit -c titanic -f /content/submission.csv -m "A test submission"

100% 2.77k/2.77k [00:06<00:00, 432B/s]
Successfully submitted to Titanic: Machine Learning from Disaster